# 👋🌍 Hello, world: Submit a Q# job to Quantinuum

In this notebook, we'll review the basics of Azure Quantum by submitting a simple *job*, or quantum program, to [Quantinuum](https://www.quantinuum.com/). We will use [Q#](https://docs.microsoft.com/azure/quantum/user-guide/) to express the quantum job.

## Submit a simple job Quantinuum using Azure Quantum
Azure Quantum provides several ways to express quantum programs. In this example we are using Q#, but note that Qiskit and Cirq are also supported. All code in this example will be written in Python and Q#.

Let's begin. When you see a code block, hover over it and click the triangle play-button to execute it. To avoid any compilation issues, this should be done in order from top to bottom.

### 1. Connect to the Azure Quantum workspace

To connect to the Azure Quantum service, call the `connect` function as seen below. This can take 30 seconds to run the first time, as the Q# environment is initializing.

In [ ]:
import qsharp.azure

targets = qsharp.azure.connect(
    resourceId = "",
    location = ""
)

We can use the resulting object to see which _targets_ are available for submission.

In [ ]:
import qsharp

print("This workspace's targets:")
for target in targets:
    print("-", target.id)

### ❕ Do you see `quantinuum.hqs-lt-s1-apival` in your list of targets? If so, you're ready to keep going.

Don't see it? You may need to add Quantinuum to your workspace to run this sample. Navigate to the **Providers** page in the portal and click **+Add** to add the Quantinuum provider. Don't worry, there's a free credits plan available.
## Quantinuum: The quantum provider
Azure Quantum partners with third-party companies to deliver solutions to quantum jobs. These company offerings are called *providers*. Each provider can offer multiple *targets* with different capabilities. See the table below for Quantinuum's targets.

| Target name | Target ID | Number of qubits | Description |
| --- | --- | --- | --- |
| H1-1 API Validator | `quantinuum.hqs-lt-s1-apival` | 12 | Quantinuum's "API validator." Use this to validate quantum programs before submitting to hardware or emulation on Quantinuum's platform. Free of cost. |
| H1 Emulator | `quantinuum.hqs-lt-s1-sim` | 12 | Quantinuum's quantum simulator. Uses a noise model that is consistent with their H1 hardware offering. |
| Quantinuum System Model: H1 Family | `quantinuum.hqs-lt` | 12 | Quantinuum's H1 family of trapped-ion quantum computers. |

For this example, we will use `quantinuum.hqs-lt-s1-apival` to avoid any costs or credit usage. If you wish to emulate or run the actual circuit, you may replace all instances of `quantinuum.hqs-lt-s1-apival` in subsequent code cells with one of the other values in the table above, but please note any costs incurred. To learn more about Quantinuum's targets, check out our [documentation](https://aka.ms/AQ/Quantinuum/Documentation).

### 2. Build the quantum program

Let's create a simple Q# program to run. This can take 30 seconds to run the first time, as the Q# environment is initializing.

In [ ]:
# We declare the function we will define in Q# as a callable so that Python recognizes the symbol.
GenerateRandomBit: any = None

In [ ]:
%%qsharp
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;

operation GenerateRandomBit() : Result {
    use target = Qubit();

    // Apply an H-gate and measure.
    H(target);
    return M(target);
}

The program you built is a simple quantum random bit generator. With Quantinuum's API validator, we will be able to confirm that the circuit is able to be run on their H1 emulator and hardware.

### 3. Submit the quantum program to Quantinuum

We will use the `execute` function to run the quantum program above on Quantinuum's `quantinuum.hqs-lt-s1-apival` target. This may take a minute or so ⏳. Your job will be packaged and sent to Quantinuum, where it will wait its turn to be run.

In [ ]:
# Set the target to quantinuum.hqs-lt-s1-apival
qsharp.azure.target("quantinuum.hqs-lt-s1-apival")

# Execute the job. We'll use 100 shots (simulated runs). Timeout is in seconds.
result = qsharp.azure.execute(GenerateRandomBit, shots=100, jobName="Generate one random bit", timeout=240)

The job ID can be used to retrieve the results later using the `output` and `status` functions or by viewing it under the **Job management** section of the portal.

### 4. Visualize the job results
You can view a histogram of the results using [`pyplot`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html):


In [ ]:
result

In [ ]:
from matplotlib import pyplot

_ = pyplot.bar(result.keys(), result.values())
pyplot.title("Result")
pyplot.ylabel("Probability")
_ = pyplot.xlabel("Measurement")

**See the histogram above? Congratulations, you've submitted a job with Azure Quantum! 👏**

### 5. Next steps
Next, you can try running a program on Quantinuum's emulation target - just replace `quantinuum.hqs-lt-s1-apival` with `quantinuum.hqs-lt-s1-sim`. Or try another sample by navigating back to the sample gallery. The same "hello world" sample can be run with different quantum providers by choosing another option in the gallery card drop-down menu. Don't worry - your work here is automatically saved.

To learn more about submitting jobs to Azure Quantum using Q#, refer to [this documentation](https://docs.microsoft.com/azure/quantum/how-to-submit-jobs?pivots=ide-python).

To learn more about job pricing, review [this documentation](https://docs.microsoft.com/azure/quantum/azure-quantum-job-costs).